In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [53]:
df_wilayah = pd.read_csv('D:\work\smeru\data\wilayah_dapodik.csv')
df_wilayah.kode_prov = '0' + df_wilayah.kode_prov.astype('str')
df_wilayah.kode_kab = '0' + df_wilayah.kode_kab.astype('str')
df_wilayah.kode_kec = '0' + df_wilayah.kode_kec.astype('str')
df_wilayah.head(3)

,prov,kode_prov,kab,kode_kab,kec,kode_kec
0,Prov. Jawa Timur,050000,Kota Surabaya,056000,Kec. Tambaksari,056020
1,Prov. Jawa Timur,050000,Kota Surabaya,056000,Kec. Wonokromo,056011
2,Prov. Jawa Timur,050000,Kota Surabaya,056000,Kec. Sawahan,056017


In [25]:
wilayah_sample = df_wilayah.head(3)

In [ ]:
# url = 'https://dapo.kemdikbud.go.id/rekap/progresSP?id_level_wilayah=3&kode_wilayah=056020&semester_id=20222&bentuk_pendidikan_id='
data_sekolah = []

for i, wil in wilayah_sample.iterrows():
    url = f'https://dapo.kemdikbud.go.id/rekap/progresSP?id_level_wilayah=3&kode_wilayah={wil.kode_kec}&semester_id=20222&bentuk_pendidikan_id='
    r = requests.get(url)
    data_sekolah = r.json()

In [30]:
url = f'https://dapo.kemdikbud.go.id/rekap/progresSP?id_level_wilayah=3&kode_wilayah={wilayah_sample.loc[0].kode_kec}&semester_id=20222&bentuk_pendidikan_id=sma'
r = requests.get(url)
data_sekolah = r.json()

In [31]:
len(data_sekolah)

7

In [33]:
url_html = f'https://dapo.kemdikbud.go.id/sekolah/{data_sekolah[0]["sekolah_id_enkrip"]}'
url_json = f'https://dapo.kemdikbud.go.id/rekap/sekolahDetail?semester_id=20222&sekolah_id={data_sekolah[0]["sekolah_id_enkrip"]}'

r_html = requests.get(url_html)
r_json = requests.get(url_json)

In [35]:
json_data = r_json.json()

In [36]:
html_soup = BeautifulSoup(r_html.content, 'html.parser')

In [116]:
data = {}

In [117]:
data['nama'] = html_soup.find('h2', {'class': 'name'}).get_text(strip=True)

In [118]:
akreditasi = html_soup.find('div', {'class': 'profile-usermenu'}).find_all('a')[2].get_text(strip=True)
data['akreditasi'] = akreditasi.split(':')[1]
kurikulum = html_soup.find('div', {'class': 'profile-usermenu'}).find_all('a')[3].get_text(strip=True)
data['kurikulum'] = kurikulum.split(':')[1]

In [119]:
data['link_sekolah_kita'] = html_soup.find('div', {'class': 'profile-userbuttons'}).find('a')['href']

In [120]:
data_utama = html_soup.find_all('div', {'class': 'panel panel-info'})

In [121]:
data_identitas = data_utama[0]
data_pelengkap = data_utama[1]
data_rinci = data_utama[2]
data_kontak = data_utama[7]

In [122]:
for identitas in data_identitas.find_all('p'):
    text = identitas.get_text()
    if text.split(':')[1].strip()=='' or text.split(':')[1].strip()=='-':
        value = None
    else:
        value = text.split(':')[1].strip()
    data[text.split(':')[0].lower().strip().replace(' ', '_')] = value


for pelengkap in data_pelengkap.find_all('p'):
    if text.split(':')[1].strip()=='' or text.split(':')[1].strip()=='-':
        value = None
    else:
        value = text.split(':')[1].strip()
    data[text.split(':')[0].lower().strip().replace(' ', '_')] = value


for rinci in data_rinci.find_all('p'):
    if text.split(':')[1].strip()=='' or text.split(':')[1].strip()=='-':
        value = None
    else:
        value = text.split(':')[1].strip()
    data[text.split(':')[0].lower().strip().replace(' ', '_')] = value

    
for kontak in data_kontak.find_all('p'):
    if text.split(':')[1].strip()=='' or text.split(':')[1].strip()=='-':
        value = None
    else:
        value = text.split(':')[1].strip()
    data[text.split(':')[0].lower().strip().replace(' ', '_')] = value

In [123]:
data

{'nama': 'SMAN 4 SURABAYA',
 'akreditasi': 'A',
 'kurikulum': 'Kurikulum 2013',
 'link_sekolah_kita': 'http://sekolah.data.kemdikbud.go.id/index.php/chome/profil/f365cfa6-ff48-4f51-8aeb-b01218566978',
 'npsn': '20532247',
 'status': 'Negeri',
 'bentuk_pendidikan': 'SMA',
 'status_kepemilikan': 'Pemerintah Daerah',
 'sk_pendirian_sekolah': '632983/65',
 'tanggal_sk_pendirian': '1900-01-01',
 'sk_izin_operasional': None,
 'tanggal_sk_izin_operasional': None}

In [126]:
data = {**data, **json_data}

# Ambil detail data di sekolah kita

In [128]:
data['link_sekolah_kita']

'http://sekolah.data.kemdikbud.go.id/index.php/chome/profil/f365cfa6-ff48-4f51-8aeb-b01218566978'

In [177]:
data_sekolah_kita = {}

In [129]:
r = requests.get(data['link_sekolah_kita'])
soup = BeautifulSoup(r.content, 'html.parser')

In [178]:
akreditasi = soup.find('div', {'class': 'box-body box-profile'}).find_all('li')[1].get_text(strip=True)
data_sekolah_kita['akreditasi'] = akreditasi.split(':')[1].strip()

In [137]:
data_utama = soup.find_all('div', {'class': 'col-xs-12 col-md-3 text-left'})

In [179]:
data_pendidikan = data_utama[0].get_text("~",strip=True)
data_pendidikan = data_pendidikan.split('~')

for i in range(len(data_pendidikan)):
    if ':' in data_pendidikan[i]:
        key = data_pendidikan[i]
        key = key.strip(' :').lower().replace(' ', '_').replace('-', '_')
        if i == len(data_pendidikan)-1:
            data_sekolah_kita[key] = None
            continue

        value = data_pendidikan[i+1]
        if ':' in value:
            value = None
        data_sekolah_kita[key] = value
        

In [180]:
data_kurikulum = data_utama[1].get_text("~",strip=True)
data_kurikulum = data_kurikulum.split('~')

for i in range(len(data_kurikulum)):
    if ':' in data_kurikulum[i]:
        if re.search(r":$", data_kurikulum[i]):
            key = data_kurikulum[i]
            key = key.strip(' :').lower().replace(' ', '_').replace('-', '_')
            if i == len(data_kurikulum)-1:
                data_sekolah_kita[key] = None
                continue
                
            value = data_kurikulum[i+1]
            if ':' in value:
                value = None
            data_sekolah_kita[key] = value
        else:
            key_value = data_kurikulum[i].split(':')
            key = key_value[0]
            key = key.strip().lower().replace(' ', '_').replace('-', '_')
            value = key_value[1]
            value = value.strip()
            data_sekolah_kita[key] = value

In [182]:
data_gedung = data_utama[2].get_text("~",strip=True)
data_gedung = data_gedung.split('~')

for i in range(len(data_gedung)):
    if ':' in data_gedung[i]:
        if re.search(r":$", data_gedung[i]):
            key = data_gedung[i]
            key = key.strip(' :').lower().replace(' ', '_').replace('-', '_')
            if i == len(data_gedung)-1:
                data_sekolah_kita[key] = None
                continue
                
            value = data_gedung[i+1]
            if ':' in value:
                value = None
            data_sekolah_kita[key] = value
        else:
            key_value = data_gedung[i].split(':')
            key = key_value[0]
            key = key.strip().lower().replace(' ', '_').replace('-', '_')
            value = key_value[1]
            value = value.strip()
            data_sekolah_kita[key] = value

In [186]:
data_sarpras = data_utama[3].get_text("~",strip=True)
data_sarpras = data_sarpras.split('~')

for i in range(len(data_sarpras)):
    if data_sarpras[i] == '*':
        continue
    if ':' in data_sarpras[i]:
        if re.search(r":$", data_sarpras[i]):
            key = data_sarpras[i]
            key = key.strip(' :').lower().replace(' ', '_').replace('-', '_')
            if i == len(data_sarpras)-1:
                data_sekolah_kita[key] = None
                continue
                
            value = data_sarpras[i+1]
            if ':' in value:
                value = None
            data_sekolah_kita[key] = value
        else:
            key_value = data_sarpras[i].split(':')
            key = key_value[0]
            key = key.strip().lower().replace(' ', '_').replace('-', '_')
            value = key_value[1]
            value = value.strip()
            data_sekolah_kita[key] = value

In [197]:
status_guru = soup.find('div', {'id': 'gurustatus'}).find_all('tr')[2:]
for sg in status_guru:
    key_value = sg.find_all('td')
    key = 'guru_'+key_value[0].get_text(strip=True).lower()
    value = key_value[1].get_text(strip=True)
    data_sekolah_kita[key] = value

In [207]:
golongan_guru = soup.find('div', {'id': 'gurugolongan'}).find_all('tr')
key_data = golongan_guru[0].find_all('td')[:-1]
value_data = golongan_guru[1].find_all('td')[:-1]

for i in range(len(key_data)):
    key = 'guru_gol_'+key_data[i].get_text(strip=True).lower()
    value = value_data[i].get_text(strip=True)
    data_sekolah_kita[key] = value

In [209]:
sertifikasi_guru = soup.find('div', {'id': 'gurusertifikasi'}).find_all('tr')[2:]
for sg in sertifikasi_guru:
    key_value = sg.find_all('td')
    key = 'guru_'+key_value[0].get_text(strip=True).lower().replace(' ', '_')
    value = key_value[1].get_text(strip=True)
    data_sekolah_kita[key] = value

In [211]:
ijazah_guru = soup.find('div', {'id': 'guruijazah'}).find_all('tr')[2:]
for sg in ijazah_guru:
    key_value = sg.find_all('td')
    key = 'guru_'+key_value[0].get_text(strip=True).lower().replace(' ', '_')
    value = key_value[1].get_text(strip=True)
    data_sekolah_kita[key] = value

umur_guru = soup.find('div', {'id': 'guruumur'}).find_all('tr')[2:]
for sg in umur_guru:
    key_value = sg.find_all('td')
    key = 'guru_'+key_value[0].get_text(strip=True).lower().replace(' - ', '_').replace(' ', '_')
    value = key_value[1].get_text(strip=True)
    data_sekolah_kita[key] = value

jeniskelamin_guru = soup.find('div', {'id': 'gurujeniskelamin'}).find_all('tr')[2:]
for sg in jeniskelamin_guru:
    key_value = sg.find_all('td')
    key = 'guru_'+key_value[0].get_text(strip=True).lower().replace('-', '_').replace(' ', '_')
    value = key_value[1].get_text(strip=True)
    data_sekolah_kita[key] = value

In [219]:
rombel_data = soup.find('div', {'id': 'rombel'}).find_all('tr')[2:]
for sg in rombel_data:
    key_value = sg.find_all('td')
    key = 'rombel_tingkat_'+key_value[0].get_text(strip=True).lower().replace(' ', '_')
    value = key_value[1].get_text(strip=True)
    data_sekolah_kita[key] = value

In [218]:
rombel_data

[<tr>
 <td>10</td>
 <td class="text-right">311</td>
 </tr>,
 <tr>
 <td>11</td>
 <td class="text-right">311</td>
 </tr>,
 <tr>
 <td>12</td>
 <td class="text-right">295</td>
 </tr>]

In [220]:
data_sekolah_kita

{'akreditasi': 'A',
 'guru': '66',
 'siswa_laki_laki': '404',
 'siswa_perempuan': '513',
 'rombongan_belajar': '27',
 'kurikulum': 'SMA 2013 MIPA',
 'penyelenggaraan': 'Sehari Penuh/5 hari',
 'manajemen_berbasis_sekolah': None,
 'semester_data': '2022/2023-2',
 'akses_internet': 'Tidak Ada',
 'sumber_listrik': 'PLN',
 'daya_listrik': '55,000',
 'luas_tanah': '0',
 'ruang_kelas': '26',
 'laboratorium': '4',
 'perpustakaan': '1',
 'sanitasi_siswa': '6',
 'guru_pns': '35',
 'guru_gtt': '16',
 'guru_gty': '2',
 'guru_honor': '13',
 'guru_gol_i': '35',
 'guru_gol_ii': '0',
 'guru_gol_iii': '20',
 'guru_gol_iv': '11',
 'guru_sertifikasi': '35',
 'guru_belum_sertifikasi': '31',
 'guru_kurang_dari_s1': '1',
 'guru_s1_atau_lebih': '56',
 'guru_data_kosong': '9',
 'guru_kurang_dari_30_tahun': '13',
 'guru_31_35_tahun': '9',
 'guru_36_40': '3',
 'guru_41_45_tahun': '6',
 'guru_46_50_tahun': '4',
 'guru_51_55_tahun': '10',
 'guru_lebih_dari_55_tahun': '21',
 'guru_laki_laki': '29',
 'guru_perempua

In [214]:
a = [1]

In [215]:
a[2:]

[]